In [1]:
from __future__ import print_function 
import os, sys
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import neuralbuild as nb
from tfomics import utils, fit

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

X_train = mnist.train.images
X_train = np.reshape(X_train, (X_train.shape[0], 28, 28, 1))
y_train = mnist.train.labels

X_valid = mnist.validation.images
X_valid = np.reshape(X_valid, (X_valid.shape[0], 28, 28, 1))
y_valid = mnist.validation.labels

X_test = mnist.test.images
X_test = np.reshape(X_test, (X_test.shape[0], 28, 28, 1))
y_test = mnist.test.labels

# get shapes
num_data, height, widht, dim = X_train.shape
input_shape=[None, height, widht, dim]
num_labels = y_train.shape[1]   # number of labels (output units)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [14]:

def model(input_shape, output_shape):

    # create model
    layer1 = {'layer': 'input',
            'input_shape': input_shape,
            }
    layer2 = {'layer': 'conv2d', 
            'num_filters': 32,
            'filter_size': (5, 5),
            'norm': 'batch',
            'activation': 'prelu',
            'dropout': 0.1,
            }
    layer3 = {'layer': 'conv2d', 
            'num_filters': 64,
            'filter_size': (5, 5),
            'norm': 'batch',
            'activation': 'prelu',
            'dropout': 0.1,
            }
    layer4 = {'layer': 'dense', 
            'num_units': 512,
            'activation': 'prelu',
            'dropout': 0.5,
            }  
    layer5 = {'layer': 'dense', 
            'num_units': num_labels,
            'activation': 'softmax',
            }

    #from tfomics import build_network
    model_layers = [layer1, layer2, layer3, layer4, layer5]

    # optimization parameters
    optimization = {"objective": "categorical",
                  "optimizer": "adam",
                  "learning_rate": 0.001,      
                  "l2": 1e-6,
                  }
    return model_layers, optimization



# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
output_shape = y_train.shape

model_layers, optimization = model(input_shape, output_shape)


In [16]:
# set output file paths
data_path = 'MNIST_data'
filename = 'MNIST'
save_path = utils.make_directory(data_path, filename)
filepath = os.path.join(save_path, filename)

# build neural network class
nnmodel = nn.NeuralNet()
nnmodel.build_layers(model_layers, optimization)
nnmodel.inspect_layers()

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, save='best', filepath=filepath)

In [17]:
sess = utils.initialize_session(nnmodel.placeholders)

In [ ]:
train = {'inputs': X_train, 'targets': y_train}
valid = {'inputs': X_valid, 'targets': y_valid}
data = {'train': train, 'valid': valid}
results = fit.train_minibatch(sess, nntrainer, data, batch_size=200, num_epochs=50, 
                              patience=20, verbose=2, shuffle=True)

Epoch 1 out of 50 
[==============================] 100.0% -- time=0s -- loss=0.41882 -- accuracy=91.36%   
  valid loss:		0.08885
  valid accuracy:	0.99492+/-0.00162
  valid auc-roc:	0.99953+/-0.00035
  valid auc-pr:		0.99682+/-0.00182
  lower cross-validation found
  saving model to:  MNIST_data/MNIST/MNIST_best.ckpt
Epoch 2 out of 50 
[==                            ] 7.3% -- time=249s -- loss=0.07755 -- accuracy=97.63%  

In [ ]:
nntrainer.set_best_parameters(sess)

test = {'inputs': X_test, 'targets': y_test}
loss, mean_vals, std_vals = nntrainer.test_model(sess, test, batch_size=128, name='test', verbose=1)